In [6]:
import torch
import sys
from torch.utils.data import DataLoader
import torchvision.transforms as T
from tqdm.notebook import tqdm
from pathlib import Path

sys.path.append('../src/')
from newspapersdataset import (
    NewspapersDataset, 
    prepare_data_for_dataloader
)
from functions import (
    from_tsv_to_list,
    collate_fn,
    predict_eval_set,
    prepare_data_for_ap,
    calculate_map
)

sys.path.append("../../")
from image_size import get_image_size  # source: https://github.com/scardine/image_size

In [7]:
parameters = {
    'channel': 1, 
    'rescale': [1000, 1000],
    'batch_size': 16,
    'shuffle': False, 
    'num_workers': 2,
    'main_dir': f'{str(Path.cwd().parent)}',
    'gpu': False,
}

In [8]:
if torch.cuda.is_available() and parameters['gpu']:
    model = torch.load('../saved_models/model.pth')
    device = torch.device(torch.cuda.current_device())
else:
    model = torch.load('../saved_models/model.pth', map_location=torch.device('cpu'))
    device = torch.device("cpu")

In [9]:
data_transform = T.Compose([
    T.Grayscale(num_output_channels=parameters['channel']),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,)),
    ])

In [22]:
# prepare val dataloader
expected_val = from_tsv_to_list(parameters['main_dir']+'/preprocessed_annotations/dev-0/expected.tsv')
in_file_val = from_tsv_to_list(parameters['main_dir']+'/preprocessed_annotations/dev-0/in.tsv')
img_paths_val = [parameters['main_dir']+'/scraped_photos/'+path for path in in_file_val]

data_val = prepare_data_for_dataloader(
    img_dir=parameters['main_dir']+'/scraped_photos/',
    in_list=in_file_val,
    expected_list=expected_val,
    bbox_format='x0y0x1y1',
    scale=parameters['rescale'],
    test=False,
    )
dataset_val = NewspapersDataset(
    df=data_val,
    images_path=img_paths_val,
    scale=parameters['rescale'],
    transforms=data_transform,
    test=False,
    )
dataloader_val = DataLoader(
    dataset_val,
    batch_size=parameters['batch_size'],
    shuffle=parameters['shuffle'],
    collate_fn=collate_fn,
    num_workers=parameters['num_workers'],
    )

In [23]:
# make prediction on test set
out_val, targ_val = predict_eval_set(
    dataloader=dataloader_val,
    model=model,
    device=device
)

100%|██████████| 14/14 [20:44<00:00, 88.91s/it]


In [24]:
# prepare data into a format adequate for AP calculation
prep_pred_val, prepr_gt_val= prepare_data_for_ap(out_val, targ_val)

In [25]:
# calculate AP for each clas and mean AP based on preprocessed model results
eval_metrics_val = calculate_map(prep_pred_val, prepr_gt_val)

In [26]:
eval_metrics_val

{'photograph': 0.76625425,
 'illustration': 0.31763086,
 'map': 0.65815836,
 'cartoon': 0.67715204,
 'editorial_cartoon': 0.3822639,
 'headline': 0.9000255,
 'advertisement': 0.81451476,
 'mAP': 0.5231611}

In [10]:
# prepare test dataloader
expected_test = from_tsv_to_list(parameters['main_dir']+'/preprocessed_annotations/test-A/expected.tsv')
in_file_test = from_tsv_to_list(parameters['main_dir']+'/preprocessed_annotations/test-A/in.tsv')
img_paths_test = [parameters['main_dir']+'/scraped_photos/'+path for path in in_file_test]

data_test = prepare_data_for_dataloader(
    img_dir=parameters['main_dir']+'/scraped_photos/',
    in_list=in_file_test,
    expected_list=expected_test,
    bbox_format='x0y0x1y1',
    scale=parameters['rescale'],
    test=False,
    )
dataset_test = NewspapersDataset(
    df=data_test,
    images_path=img_paths_test,
    scale=parameters['rescale'],
    transforms=data_transform,
    test=False,
    )
dataloader_test = DataLoader(
    dataset_test,
    batch_size=parameters['batch_size'],
    shuffle=parameters['shuffle'],
    collate_fn=collate_fn,
    num_workers=parameters['num_workers'],
    )

In [11]:
# make prediction on test set
out_test, targ_test = predict_eval_set(
    dataloader=dataloader_test,
    model=model,
    device=device
)

100%|██████████| 45/45 [1:08:17<00:00, 91.06s/it]


In [17]:
# prepare data into a format adequate for AP calculation
prep_pred_test, prepr_gt_test = prepare_data_for_ap(out_test, targ_test)

In [20]:
# calculate AP for each clas and mean AP based on preprocessed model results
eval_metrics_test = calculate_map(prep_pred_test, prepr_gt_test)

In [21]:
eval_metrics_test

{'photograph': 0.76897764,
 'illustration': 0.4193151,
 'map': 0.65971047,
 'cartoon': 0.72911716,
 'editorial_cartoon': 0.5587153,
 'headline': 0.903047,
 'advertisement': 0.8602427,
 'mAP': 0.56490856}